In [4]:
import sys
sys.path.append("../")
import rg
from rg.graph import incidence_matrix
from rg.graph import ring_diagram ,show_spanning_trees
from rg.graph import circuits, set_circuit_flow,set_source_sink_flow,apply_flow
from rg.graph import scalar_propagator
#for inline dev
import numpy as np
from sympy import *

## work in progress

What is the best method for determining the edge basis. Cycles can be used. 
- Important to partition cycles into positive and negative frequencies
- Import to attribute to each edge the correct edge basis

### from the constraints at vertices, we try to construct an edge basis 
 - need to be careful about Sympy interface because I am not sure about some things yet

In [5]:
sunset = incidence_matrix(3,2,
                       [ [0,1], [1,0], [0,1]  ],
                       external_vertices=[0,1])

L = sunset.first_betti_number
maps = sunset.edges_to_loop_basis()
#these are the ids of each edge in the incidence matrix and their coefficients on the edge basis
#if there are two loops and two externals each edge takes a vector: [L1|L2|EX1|EX2]
maps

{0: array([-1, -1,  0, -1]), 1: array([0, 1, 0, 0]), 2: array([1, 0, 0, 0])}

In [6]:
#given an edge basis and a known loop number, create the propagator for each edge - here we know to use the first L as the loop basis
scalar_propagator(maps[0][:L], maps[0][L:])

$\frac{1}{D_{a} \left(k'_{1} + k_{0} + k_{1}\right)^{2} - i \omega'_1 + m_{a}^{2} - i \omega_{0} - i \omega_{1}}$

In [7]:
ring_diagram(sunset)

### all the propagators look like this

In [ ]:
rg._product_([scalar_propagator(maps[i][:L], maps[i][L:]).value for i in maps.keys()])

In [ ]:
#below is displayed with the source sink flow used to solve constraints and not the loop flow...
#but we can see from the propagators that e0 and e1 flow in the *some* direction and e1 flows opposite for each loop variable
#it is important that the loop variables are represented with opposite signs for the set of propagaors
ring_diagram(sunset, options={"show_labels"})

In [ ]:
apply_flow(sunset,set_circuit_flow)
ring_diagram(sunset, options={"show_labels"})

### example of the shorthand 
- convention must be known but useful to create a larger integral from matrix of coefficients

In [ ]:
mat = [[0,1,0,1,1,1],
       [0,-1,0,0,2,2]]
scalar_propagator.from_basis_vector(mat,2)

# A first look at the incidence matrix class

In [ ]:
itest = incidence_matrix.__sample__()
itest

In [ ]:
INC = incidence_matrix.__sample__()
#INC.flip_edges(1)
D = ring_diagram(INC, options={ "show_labels":True, }) 
D

In [ ]:
all_circuits=  circuits(INC,sort=True)
all_circuits

# now take a look at one of the spanning rees

In [ ]:
#show_spanning_trees(INC)

In [ ]:
tree = INC.spanning_trees.iloc[-1].values
tree_cuts = INC.edge_complement(tree)
print(tree_cuts)
C = ring_diagram(INC, options={"show_labels" :True,  "cut_edges": tree_cuts}) 
C#._repr_html_()

# Edge Orientations
given cicuits, we can actually rank them by closeness to root and then we can incrementally orientate them in different ways

(1) Choose a source and sink delegate on each circuit and have allow flow towards the sink

(2) Respect loop momentum

In [ ]:
#todo the last circuit is degenerate
INC = incidence_matrix.__sample__()
all_circuits = circuits(INC,INC.spanning_trees.iloc[0].values)
print(all_circuits)

# set default circuit flow

In [ ]:
INC = incidence_matrix.__sample__()
#todo technicall need to reload cicuts but they are ok
all_circuits = circuits(INC,INC.spanning_trees.iloc[0].values)
circ = all_circuits[1]
#print(circ)
res = set_circuit_flow(INC,circ)
#print(res)
#print("edge comp", circ)
ring_diagram(INC, options={"show_labels" :True,  "cut_edges": INC.edge_complement(circ)}) 

# circuit source sink flow

In [ ]:
INC = incidence_matrix.__sample__()
#todo technicall need to reload cicuts but they are ok
all_circuits = circuits(INC,INC.spanning_trees.iloc[0].values)
circ = all_circuits[1]
#print(circ)
res = set_source_sink_flow(INC,circ)
print("source,sink",res)
#print(res)
#print("edge comp", circ)
ring_diagram(INC, options={"show_labels" :True,  "cut_edges": INC.edge_complement(circ)}) 

# all together now...

cicuits are ordered based on the average depth of vertices in the cicuit. Outer rings score best

We process all cicuits in that order

Suppose we dont remember anything from the last processed circuit...

In [ ]:
apply_flow(INC,source_sink_flow)
ring_diagram(INC, options={"show_labels" :True}) 

# Other examples

In [ ]:
simp = incidence_matrix(2,2,
                       [ [0,1], [1,0]  ],
                       external_vertices=[0,1])

sunset = incidence_matrix(3,2,
                       [ [0,1], [1,0], [0,1]  ],
                       external_vertices=[0,1])

bog = incidence_matrix(7,6,
                       [ [0,1], [1,3], [0,2], [3,2], [3,5], [2,4], [4,5]  ],
                       external_vertices=[0,1,4,5])

#get cicuits using any spanning tree and produce a sink flow ordering
set_source_sink_flow(simp,circuits(simp)[0])
#simp.constraint_system(True)
ring_diagram(simp)

In [ ]:
show_spanning_trees(simp)

In [ ]:
simp = incidence_matrix(3,3,
                       [ [0,1], [1,2], [2,0]  ],
                       external_vertices=[0,1])
ring_diagram(simp)

In [ ]:
show_spanning_trees(simp)

In [ ]:
simp = incidence_matrix(4,4,
                       [ [0,1], [1,2], [2,3], [3,0]  ],
                       external_vertices=[0,1])
ring_diagram(simp)

In [ ]:
#TODO: visual style for and implementation of multiple external legs
#perhaps an option for simple diagrams to force certain types of loops and ignore hierarchy

#save svg todo find save html tp pdf solution
#with open ("test.svg", "w") as f:
#    f.write(ring_diagram(sunset, options={"show_labels"})._repr_html_())